# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [80]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import holoviews as hv
from pprint import pprint
# Import API key
from api_keys import geoapify_key


#Ignore warning
import warnings
warnings.filterwarnings("ignore")

In [81]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,flin flon,54.7682,-101.8650,-11.01,94,15,1.80,CA,1700195224
1,1,acarau,-2.8856,-40.1200,26.43,80,73,6.89,BR,1700195224
2,2,shimoda,34.6667,138.9500,19.20,82,100,10.34,JP,1700195225
3,3,margaret river,-33.9500,115.0667,23.36,41,0,3.52,AU,1700195225
4,4,buka,40.8108,69.1986,15.49,17,100,1.13,UZ,1700195225


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [82]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    color = "City",
    fill_alpha = 0.5,
    title = "Plot showing cities from city_data_df",
    xlabel = "Latitude",
    ylabel = "Longitude"
)


# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [83]:
# Narrow down cities that fit criteria and drop any results with null values
# Establish the parameter 
temp_min = 20
temp_max = 35
win_speed_max = 5
cloudiness_max = 30
humidity_max = 50

# Sort the DataFrame for weather conditions
ideal_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= temp_min) &
                                   (city_data_df["Max Temp"] <= temp_max) &
                                   (city_data_df["Wind Speed"] <= win_speed_max) &
                                   (city_data_df["Cloudiness"] <= cloudiness_max) &
                                   (city_data_df["Humidity"] <= humidity_max)]

# Drop any rows with null values
ideal_cities_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,margaret river,-33.9500,115.0667,23.36,41,0,3.52,AU,1700195225
21,21,tazacorte,28.6290,-17.9293,25.08,29,0,3.09,ES,1700195228
120,120,blackmans bay,-43.0167,147.3167,21.07,48,14,0.98,AU,1700195249
160,160,lospalos,-8.5186,127.0025,32.63,40,14,2.92,TL,1700195258
171,171,new norfolk,-42.7826,147.0587,21.72,48,23,3.91,AU,1700195260
199,199,sanchor,24.7553,71.7722,25.00,26,3,1.20,IN,1700195264
210,210,tchintabaraden,15.8969,5.7985,22.00,25,0,4.33,NE,1700195266
238,238,carinhanha,-14.3047,-43.7650,26.52,49,6,2.18,BR,1700195270
252,252,kidal,18.4411,1.4078,21.48,22,0,4.85,ML,1700195178
371,371,kedougou,12.5579,-12.1743,22.66,26,3,0.77,SN,1700195292


### Step 3: Create a new DataFrame called `hotel_df`.

In [84]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
3,margaret river,AU,-33.9500,115.0667,41,
21,tazacorte,ES,28.6290,-17.9293,29,
120,blackmans bay,AU,-43.0167,147.3167,48,
160,lospalos,TL,-8.5186,127.0025,40,
171,new norfolk,AU,-42.7826,147.0587,48,
199,sanchor,IN,24.7553,71.7722,26,
210,tchintabaraden,NE,15.8969,5.7985,25,
238,carinhanha,BR,-14.3047,-43.7650,49,
252,kidal,ML,18.4411,1.4078,22,
371,kedougou,SN,12.5579,-12.1743,26,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [85]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"

params = {
    "categories":categories,
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
        latitude = hotel_df.loc[index, "Lat"]
        longitude = hotel_df.loc[index, "Lng"]
    
    
# Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
params["filter"] = f"circle:{longitude},{latitude},{radius}"
params["bias"] = f"proximity:{longitude},{latitude}"
    
# Set base URL
base_url = "https://api.geoapify.com/v2/places"


# Make and API request using the params dictionaty
name_address = requests.get(base_url, params=params)
    
# Convert the API response to JSON format
name_address = name_address.json()
    
# Grab the first hotel from the results and store the name in the hotel_df DataFrame
try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
# Log the search results
print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
humaita - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
3,margaret river,AU,-33.9500,115.0667,41,
21,tazacorte,ES,28.6290,-17.9293,29,
120,blackmans bay,AU,-43.0167,147.3167,48,
160,lospalos,TL,-8.5186,127.0025,40,
171,new norfolk,AU,-42.7826,147.0587,48,
199,sanchor,IN,24.7553,71.7722,26,
210,tchintabaraden,NE,15.8969,5.7985,25,
238,carinhanha,BR,-14.3047,-43.7650,49,
252,kidal,ML,18.4411,1.4078,22,
371,kedougou,SN,12.5579,-12.1743,26,


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [86]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo_alpha=True,  # Use geo_alpha instead of geo in some versions
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    scale=2.0,
    color="City",
    fill_alpha=0.5,
    title="Plot showing ideal cities for vacation",
    xlabel="Latitude",
    ylabel="Longitude",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)